### Data Processing

#### The purpose of this notebook is to read in inintial run data, investigate features, and perform preprocessing for modeling.

In [5]:
import polars as pl

raw_data_path = "../data-raw/run_data_1-16-26.csv"
raw_data = pl.read_csv(raw_data_path)
raw_data.head()

timestamp,accel_x,accel_y,accel_z,speed_mps
f64,f64,f64,f64,f64
1.7686e9,0.202988,-0.571716,-0.608765,0.0
1.7686e9,0.259293,-0.597717,-0.709015,0.0
1.7686e9,0.290527,-0.555283,-0.788055,0.0
1.7686e9,0.174026,-0.39856,-0.739792,0.0
1.7686e9,0.099594,-0.373596,-0.715012,0.0


In [6]:
from gait_analytics.preprocess import process_gait_data
from gait_analytics.visualization import plot_data

df_preprocessed = process_gait_data(raw_data)
plot_data(df_preprocessed)

In [7]:
df_preprocessed.head()

timestamp,accel_x,accel_y,accel_z,speed_mps,accel_magnitude,is_step,cadence_steps_per_minute,accel_magnitude_smoothed,target_speed_mps_smoothed
f64,f64,f64,f64,f64,f64,bool,f64,f64,f64
1.7686e9,-0.45491,0.162384,-0.308426,0.0,0.573096,false,180.0,0.499111,0.7956
1.7686e9,-0.440277,0.162308,-0.150299,0.0,0.492725,false,180.0,0.483114,0.816
1.7686e9,-0.33432,0.034775,-0.000259,0.0,0.336124,false,180.0,0.502502,0.8364
1.7686e9,-0.428482,-0.061768,-0.045181,0.0,0.435263,false,180.0,0.581293,0.8568
1.7686e9,-0.55397,-0.204468,-0.327637,0.0,0.675304,false,180.0,0.695047,0.8772


In [8]:
df_preprocessed.write_parquet("../data-processed/run_data_1-16-26_processed.parquet")